# New Section

\begin{equation}
T_{robot} =
    R_z(q_1) * 
    [T_xT_yR_x] *
    R_y(q_2 + \Delta q_2) *
    [T_xR_xR_z] * \
    R_y(q_3 + \Delta q_3) * 
    [T_xT_zR_z] *
    R_x(q_4 + \Delta q_4) * \
    [T_yT_zR_z] *
    R_y(q_5 + \Delta q_5) * 
    [T_zR_z] *
    R_x(q_6)
\end{equation}

In [ ]:
import pandas as pd
import sympy as sp
import numpy as np

In [ ]:
df = pd.read_csv('Combined_with_force_1_iteration.csv', delimiter=';')
# df = df.drop(axis=1, index=31)
df.head(2)

,timestamp,"Fx, N",Fy N,Fz N,"Tx, Nm","Ty, Nm","Tz, Nm","q0, deg","q1, deg","q2, deg",...,"p3x, mm","p3y, mm","p3z, mm","p4x, mm","p4y, mm","p4z, mm","p5x, mm","p5y, mm","p5z, mm",Unnamed: 31
0,2021-10-18 15:23:21.165886,-140.10000,-673.65667,31.17807,34.93760,-1.67260,-65.29833,-25.00,0.00,0.00,...,1217.326,-488.081,1418.160,1243.679,-428.370,1326.219,1238.507,-437.728,1237.901,NaN
1,2021-10-18 15:23:55.586442,134.20267,-652.36333,-91.26467,98.17017,6.21358,70.66700,-9.72,7.73,-7.86,...,1482.704,-181.030,1268.388,1479.981,-114.342,1177.520,1465.530,-122.670,1090.120,NaN


In [ ]:
df.tail(2)

,timestamp,"Fx, N",Fy N,Fz N,"Tx, Nm","Ty, Nm","Tz, Nm","q0, deg","q1, deg","q2, deg",...,"p3x, mm","p3y, mm","p3z, mm","p4x, mm","p4y, mm","p4z, mm","p5x, mm","p5y, mm","p5z, mm",Unnamed: 31
49,2021-10-18 16:02:54.890336,341.87133,570.77367,244.730,-103.700,12.32483,169.46567,45.0,28.38,-20.51,...,1277.277,1378.496,916.814,1207.387,1400.968,831.228,1192.478,1371.066,748.786,NaN
50,2021-10-18 16:03:47.253293,521.34367,-412.38667,196.719,-53.047,17.47467,260.83767,45.0,-48.83,-10.26,...,334.617,436.162,868.488,275.909,469.819,778.284,271.779,450.681,691.506,NaN


In [ ]:
x, y, z = sp.symbols('x, y, z')
q1, q2, q3, q4, q5, q6 = sp.symbols('q_1, q_2, q_3, q_4, q_5, q_6')
l1, l2, l3, l4, l5 = sp.symbols('l_1, l_2, l_3, l_4, l_5')


Rx = sp.Matrix([[1, 0, 0, 0],
               [0, sp.cos(q1), -sp.sin(q1), 0],
               [0, sp.sin(q1), sp.cos(q1), 0], 
                [0, 0, 0, 1]])
Ry = sp.Matrix([
                [sp.cos(q1), 0, sp.sin(q1), 0],
                [0, 1, 0, 0],
                [-sp.sin(q1), 0, sp.cos(q1), 0],
                [0, 0, 0, 1]])
Rz = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0, 0], 
               [sp.sin(q1), sp.cos(q1), 0, 0],
               [0, 0, 1, 0],
                [0, 0, 0, 1]])

Tx = sp.Matrix([
                [1, 0, 0, x],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]]
  )
Ty = sp.Matrix([
                [1, 0, 0, 0],
                [0, 1, 0, y],
                [0, 0, 1, 0],
                [0, 0, 0, 1]]
  )
Tz = sp.Matrix([
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, z],
                [0, 0, 0, 1]]
  )

In [ ]:
def FK(q=None, links=None):
    T = Rz@Tx.subs(x, l1)@Ty.subs(y, l1)@Rx@\
    Ry.subs(q1, q2)@Tx.subs(x, l2)@Rx.subs(q1, q2)@Rz.subs(q1, q2)@\
    Ry.subs(q1, q3)@Tx.subs(x, l3)@Tz.subs(z, l3)@Rz.subs(q1, q3)@\
    Rx.subs(q1, q4)@Ty.subs(y, l4)@Tz.subs(z, l4)@Rz.subs(q1, q4)@\
    Ry.subs(q1, q5)@Tz.subs(z, l5)@Rz.subs(q1, q5)@ Rx.subs(q1, q6)

    return T

In [ ]:
T = FK()
T.shape

(4, 4)

In [ ]:
P = T.col(-1)
P.shape

(4, 1)

In [ ]:
Jacobian = []

qs = [q2, q3, q4, q5]

for f in P[:3]:
    # x, y, z
    Jacobian.append([sp.diff(f,q) for q in qs])

Jacobian = sp.Matrix(Jacobian)

Jacobian.shape

(3, 4)

In [ ]:
Jacobian

Matrix([
[                                                                                                                                                                                                                                                                                                                                      l_2*(-sin(q_1)**2*cos(q_2) - sin(q_2)*cos(q_1)) + l_3*((-(-sin(q_1)**2*sin(q_2) + cos(q_1)*cos(q_2))*cos(q_2) + (sin(q_1)**2*cos(q_2) + sin(q_2)*cos(q_1))*sin(q_2) - sin(q_1)*cos(q_1)*cos(q_2))*sin(q_3) + (((sin(q_1)**2*cos(q_2) + sin(q_2)*cos(q_1))*sin(q_2) - sin(q_1)*cos(q_1)*cos(q_2))*cos(q_2) - (-sin(q_1)**2*sin(q_2) + cos(q_1)*cos(q_2))*sin(q_2) + (-sin(q_1)**2*cos(q_2) - sin(q_2)*cos(q_1))*cos(q_2) + ((-sin(q_1)**2*sin(q_2) + cos(q_1)*cos(q_2))*sin(q_2) + (sin(q_1)**2*cos(q_2) + sin(q_2)*cos(q_1))*cos(q_2) + sin(q_1)*sin(q_2)*cos(q_1))*sin(q_2))*cos(q_3)) + l_3*(((-sin(q_1)**2*sin(q_2) + cos(q_1)*cos(q_2))*cos(q_2) - (sin(q_1)**2*cos(q_2) + sin(q_2)*

In [ ]:
ls = [l1, l2, l3, l4, l5]
links_length = [0.4, 0.5, 0.5, 0.2, 0.1]
joints = ['q0, deg', 'q1, deg', 'q2, deg', 'q3, deg', 'q4, deg', 'q5, deg']
ps = ['p0x, mm', 'p0y, mm', 'p0z, mm', 'p1x, mm', 'p1y, mm', 'p1z, mm',
       'p2x, mm', 'p2y, mm', 'p2z, mm']
qs = [q1, q2, q3, q4, q5, q6]
J_lambda = sp.lambdify(qs+ls, Jacobian, 'numpy')
pi = 0

left = np.zeros((4,4))
right = np.zeros((4,1))

_all = len(df)
for i in range(_all): # i = 1 to m
    _q1 = df[joints[0]][i]
    _q2 = df[joints[1]][i]
    _q3 = df[joints[2]][i]
    _q4 = df[joints[3]][i]
    _q5 = df[joints[4]][i]
    _q6 = df[joints[5]][i]

    args = [_q1, _q2, _q3, _q4, _q5, _q6]+links_length
    # print(args)
    J = J_lambda(*args)

    a = df[ps[0]][i]
    b = df[ps[1]][i]
    c = df[ps[2]][i]
    d = df[ps[3]][i]
    e = df[ps[4]][i]
    f = df[ps[5]][i]
    g = df[ps[6]][i]
    h = df[ps[7]][i]
    ki = df[ps[8]][i]


    p0 = np.array([[a, b, c]]).T
    p1 = np.array([[d, e, f]]).T
    p2 = np.array([[g, h, ki]]).T

    # print(J)
    # sp.Matrix(p0)
    # raise Exception
    left = left + (3*(J.T@J))
    sub = J.T@p0
    # print(sub)
    sub += J.T@p1
    sub += J.T@p2

    right = right + sub

    # print(f"{i}/{_all}")

pi = np.linalg.pinv(left)@right
# sp.Matrix(p0)
print(pi)

[[   4.44640106]
 [-285.79494376]
 [-350.65989764]
 [ 270.07650501]]
